In [ ]:
import numpy as np
import pandas as pd
import requests


In [34]:
# read data
df_proposals_votes = pd.read_csv("./raw_data/2024-07-01_optimism_proposals_votes.csv")
# unify proposal_id to hexdecimal
df_proposals_votes.proposal_id = df_proposals_votes.proposal_id.apply(lambda x: str(hex(int(x[18:]))) if 'Proposal|Proposal|' in x else x)
# fill na otherwise groupby will drop rows
df_proposals_votes = df_proposals_votes.fillna('N/A')

/var/folders/db/bjx_3t8n2d73cp93pby9k97r0000gn/T/ipykernel_27113/4031484080.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_proposals_votes = pd.read_csv("./raw_data/2024-07-01_optimism_proposals_votes.csv")


In [44]:

# df_summary = df_proposals_votes[['vote_type','proposal_id','title','voted_choice','voting_power']].copy()

# # df_summary.loc[df_summary['vote_type']=='OFF_CHAIN','voting_power'] = 1

# # df_summary.vote_type = df_summary.vote_type.map({'OFF_CHAIN':'citizen_house','ON_CHAIN':'token_house'})

# df_summary = df_summary.groupby(['vote_type','proposal_id','title','voted_choice'],as_index=False).sum()

# df_summary = df_summary.sort_values(by=['proposal_id', 'voting_power'],ascending=[True,False])

# df_summary = df_summary.reset_index(drop=True)


# Token House Votes

In [2]:
# df_Top1000Delegates = pd.read_csv("./Top1000Delegates.csv")
# df_vote_cast = pd.read_csv("./vote_cast.csv")
# df_proposals = pd.read_csv("./proposals.csv")

In [50]:
df_onchain = df_proposals_votes.query('vote_type == "ON_CHAIN"').reset_index(drop=True)

# Step 1: Sort the DataFrame by 'proposal_id' and 'voting_power' in descending order
df_sorted = df_onchain.sort_values(by=['proposal_id', 'voting_power'], ascending=[True, False])

# Step 2: Group by 'proposal_id' and select the top 200 for each group
df_top_200_onchain = df_sorted.groupby('proposal_id').head(200).reset_index(drop=True)


In [154]:
df_top_200_onchain.to_csv("./token_house_proposals_votes.csv")

# Citizen House Votes

In [50]:
# df_offchain = df_proposals_votes.query('vote_type == "OFF_CHAIN"').reset_index(drop=True)

# # Step 1: Sort the DataFrame by 'proposal_id' and 'voting_power' in descending order
# df_sorted = df_offchain.sort_values(by=['proposal_id', 'voting_power'], ascending=[True, False])

# # Step 2: Group by 'proposal_id' and select the top 200 for each group
# df_top_200_offchain = df_sorted.groupby('proposal_id').head(200).reset_index(drop=True)
# # df_top_200_offchain.to_csv('./citizen_house_proposals_votes.csv')


In [1]:
import requests
import pandas as pd

# df_citizen_snapshot_proposal = pd.read_json('citizen_snapshot_proposals.json')

url = "https://hub.snapshot.org/graphql?operationName=Proposals&query=query%20Proposals%20%7B%0A%20%20proposals%20(%0A%20%20%20%20first%3A%20100%2C%0A%20%20%20%20skip%3A%200%2C%0A%20%20%20%20where%3A%20%7B%0A%20%20%20%20%20%20space_in%3A%20%5B%22citizenshouse.eth%22%5D%2C%0A%20%20%20%20%20%20state%3A%20%22closed%22%0A%20%20%20%20%7D%2C%0A%20%20%20%20orderBy%3A%20%22created%22%2C%0A%20%20%20%20orderDirection%3A%20desc%0A%20%20)%20%7B%0A%20%20%20%20id%0A%20%20%20%20title%0A%20%20%20%20body%0A%20%20%20%20choices%0A%20%20%20%20start%0A%20%20%20%20end%0A%20%20%20%20snapshot%0A%20%20%20%20state%0A%20%20%20%20author%0A%20%20%20%20space%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%20%20name%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D"
response = requests.get(url)
df_citizen_snapshot_proposal=pd.DataFrame(response.json()['data']['proposals'])
df_citizen_snapshot_proposal=df_citizen_snapshot_proposal.rename(columns={'id':'proposal_id'})


In [6]:
df_citizen_snapshot_proposal_votes = pd.DataFrame()

for id in df_citizen_snapshot_proposal.proposal_id.to_list():
    url = f"https://hub.snapshot.org/graphql?operationName=Votes&query=query%20Votes%20%7B%0A%20%20votes%20(%0A%20%20%20%20first%3A%201000%0A%20%20%20%20skip%3A%200%0A%20%20%20%20where%3A%20%7B%0A%20%20%20%20%20%20proposal%3A%20%22{id}%22%0A%20%20%20%20%7D%0A%20%20%20%20orderBy%3A%20%22created%22%2C%0A%20%20%20%20orderDirection%3A%20desc%0A%20%20)%20%7B%0A%20%20%20%20id%0A%20%20%20%20voter%0A%20%20%20%20created%0A%20%20%20%20proposal%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%7D%0A%20%20%20%20choice%0A%20%20%20%20space%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A"
    response = requests.get(url)
    temp = response.json()['data']['votes']
    for t in range(len(temp)):
        temp[t]['proposal_id'] = temp[t]['proposal']['id']
        temp[t]['space_id'] = temp[t]['space']['id']
    temp_df = pd.DataFrame(temp)
    df_citizen_snapshot_proposal_votes = pd.concat([df_citizen_snapshot_proposal_votes,temp_df])

df_citizen_snapshot_proposal_votes = pd.merge(df_citizen_snapshot_proposal_votes,df_citizen_snapshot_proposal,how="left",on='proposal_id')

df_citizen_snapshot_proposal_votes['voted_choice'] = df_citizen_snapshot_proposal_votes.apply(lambda row: row['choices'][row['choice']-1], axis=1)
df_citizen_snapshot_proposal_votes['voting_power'] = 1
df_citizen_snapshot_proposal_votes['vote_type'] = 'citizen_house'
df_citizen_snapshot_proposal_votes = df_citizen_snapshot_proposal_votes.rename(columns={'voter':'address'})
df_citizen_snapshot_proposal_votes['address'] = df_citizen_snapshot_proposal_votes['address'].str.lower()


In [57]:
df_citizen_snapshot_proposal_votes.to_csv('./citizen_house_proposals_votes2.csv')

In [59]:

np.intersect1d(df_citizen_snapshot_proposal_votes.address.values, 
                              df_top_200_onchain.address.values)

array(['0x00409fc839a2ec2e6d12305423d37cd011279c09',
       '0x0331969e189d63fbc31d771bb04ab44227d748d8',
       '0x1de2a056508e0d0dd88a88f1f5cdf9cfa510795c',
       '0x2b888954421b424c5d3d9ce9bb67c9bd47537d12',
       '0x3db5b38ef4b433d9c6a664bd35551be73313189a',
       '0x429f9ada43e9f345cbb85ec88681bb70df808892',
       '0x45a10f35befa4ab841c77860204b133118b7ccae',
       '0x48a63097e1ac123b1f5a8bbffafa4afa8192fab0',
       '0x585639fbf797c1258eba8875c080eb63c833d252',
       '0x5d36a202687fd6bd0f670545334bf0b4827cc1e2',
       '0x66946def4ba6153c500d743b7a5febfc1654d6bd',
       '0x69dc230b06a15796e3f42baf706e0e55d4d5eaa1',
       '0x69e271483c38ed4902a55c3ea8aab9e7cc8617e5',
       '0x6eda5acaff7f5964e1ecc3fd61c62570c186ca0c',
       '0x75536cf4f01c2bfa528f5c74ddc1232db3af3ee5',
       '0x7899d9b1181cbb427b0b1be0684c096c260f7474',
       '0x7fc80fad32ec41fd5cfcc14eee9c31953b6b4a8b',
       '0x839395e20bbb182fa440d08f850e6c7a8f6f0780',
       '0x8f07bc36ff569312fdc41f3867d80bbd2fe9

# Token + Citizen House Summary

In [41]:
df_summary = df_proposals_votes[['vote_type','proposal_id','title','voted_choice','voting_power']].copy()
df_summary = df_summary[df_summary['vote_type']=='OFF_CHAIN']
df_summary['vote_type'] = 'token_house'

df_summary = pd.concat([df_summary, df_citizen_snapshot_proposal_votes[['vote_type','proposal_id','title','voted_choice','voting_power']]], ignore_index=True)

df_summary = df_summary.groupby(['vote_type','proposal_id','title','voted_choice'],as_index=False).sum()

df_summary = df_summary.sort_values(by=['vote_type','proposal_id', 'voting_power'],ascending=[True,True,False])

df_summary = df_summary.reset_index(drop=True)

In [44]:
df_summary.to_csv('./voting_results_summary.csv')


# Farcaster

In [94]:
addresses = df_top_200_onchain.address.to_list()

In [ ]:
import requests

# Replace with your actual Farcaster API base URL
BASE_URL = "https://hub.pinata.cloud/v1/"

# List of Ethereum addresses to check
# addresses = [
#     "0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045",
#     "0x5e349eca2dc61abcd9dd99ce94d04136151a09ee",
#     # Add more addresses as needed
# ]

# Function to get Farcaster FID from Ethereum address
def get_farcaster_fid(address):
    response = requests.get(f"{BASE_URL}/verificationsByAddress", params={"address": address})
    if response.status_code == 200:
        data = response.json()
        if data.get("verifications"):
            return data["verifications"][0]["fid"]
    return None

# Function to get followers of a Farcaster account
def get_followers(fid):
    response = requests.get(f"{BASE_URL}/linksByFid", params={"fid": fid, "link_type": "follow"})
    if response.status_code == 200:
        data = response.json()
        return [msg["data"]["linkBody"]["targetFid"] for msg in data.get("messages", [])]
    return []

# Function to get followings of a Farcaster account
def get_followings(fid):
    response = requests.get(f"{BASE_URL}/linksByTargetFid", params={"target_fid": fid, "link_type": "follow"})
    if response.status_code == 200:
        data = response.json()
        return [msg["data"]["fid"] for msg in data.get("messages", [])]
    return []

# Main script
farcaster_accounts = {}

for address in addresses:
    fid = get_farcaster_fid(address)
    if fid:
        farcaster_accounts[address] = {
            "fid": fid,
            "followers": get_followers(fid),
            "followings": get_followings(fid)
        }
    else:
        farcaster_accounts[address] = None

# Print results
for address, data in farcaster_accounts.items():
    if data:
        print(f"Address: {address} (FID: {data['fid']})")
        print(f"Followers FIDs: {data['followers']}")
        print(f"Followings FIDs: {data['followings']}")
    else:
        print(f"Address: {address} has no Farcaster account.")


In [98]:
for address, data in farcaster_accounts.items():
    if data:
        print(f"Address: {address} (FID: {data['fid']})")
        print(f"Followers FIDs: {data['followers']}")
        print(f"Followings FIDs: {data['followings']}")
    # else:
    #     print(f"Address: {address} has no Farcaster account.")

In [ ]:
# https://crimson-tiny-urial-232.mypinata.cloud
# curl --request GET \
#   --url 'https://hub.pinata.cloud/v1/castsByFid?fid=6023&pageSize=10&reverse=true'
